# Pipeline necessary to obtain a map of the Moon

It should become something like:
1. Filter TOD
2. Create a map from it
2. Use Gaussian matched filter on the map to get the Moon's position
3. Fit a synthesised beam shape centered on Moon position to see if rotation needed (free params: values of sb shape, rotation)
4. Use the true (simulated) synthbeam, multiply it by Moon spectrum, create fake filtered TOD from it and fit the Moon spectrum (free param: Moon spectrum)

## Imports and notebook configuration

In [ ]:
# Allows the modification of the files imported without having to restart kernel or re-import them
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# to have interactive plots : pip install ipympl
%matplotlib ipympl

### General imports
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pickle

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

### Astropy configuration
import astropy.units as u

#### QUBIC IMPORT
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument
import qubic.lib.Calibration.Qfiber as ft

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
import fitting as fit
import time_domain_tools as tdt
import pipeline_moon_plotting as pmp
import pipeline_moon_functions as pmf


## Dataset and Observing site
in the case of the July 2022 Moon observations, they were from Salta CNEA Regional and the UTC offset was -3 hours.

### Location of the raw TOD files
Beware this will have to be changed for each one's configuration

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'

### Observation date and corresponding file

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*')
print(dirs)
datadir = dirs[0]


### Observing Site

In [ ]:
Salta_CNEA = {'lat':-24.731358*u.deg,
              'lon':-65.409535*u.deg,
              'height':1152*u.m,
              'UTC_Offset':-3*u.hour}
Obs_Site = Salta_CNEA

## Now the big loop on the TESs to make coadded images of the Moon data.
Here we make coadded maps in a "local" coordinate system that follows the Azimuth and Elevation of the Moon.

In [ ]:
# Initial position for gnomomic projection
myrotinit = np.array([-1, -4.])
allTESNum = np.arange(256) + 1

In [ ]:
# %%script echo skipping

nside = 256
azqubic = 116.4

### Check if the data has already been read from disk or not
try:
    data_TOD
except NameError:
    # data = read_data(datadir, remove_t0=False)
    data_TOD = None

oneTESNum = [152] #19, 28, 152, 153, 154, 238
# allTESNum = [93] # 227 is off? # chercher le script de Belen, qui enlève les flux jumps
onemap, data_TOD, center, newazt, newelt, scantype = pmf.make_coadded_maps(datadir, Obs_Site, oneTESNum, data=data_TOD, 
                                       nside=nside, doplot=True, az_qubic=azqubic, parallel=False, speedmin=0., check_back_forth=False)



In [ ]:
print(np.shape(onemap))

In [ ]:
# %%script echo skipping

plt.figure()
n0 = 1
for i in range(len(oneTESNum)):
    ok = onemap[i,:] != hp.UNSEEN
    if (i%(n0**2))==0 and i > 0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = onemap[i, ok] -tdt.get_mode(onemap[i,ok])
    mm[~ok] = hp.UNSEEN

    # vline = hp.ang2pix()

    # hp.gnomview(mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=1.2e4, 
    #             title=allTESNum[i], rot=center)
    # onemap_arr = hp.gnomview(onemap[i], reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=1.2e4, 
    #             title=allTESNum[i], rot=center, return_projected_map=True).data
    onemap_arr = hp.gnomview(onemap[i], reso=10, sub=(n0,n0,(i%(n0**2))+1), 
                title=allTESNum[i], rot=center, return_projected_map=True).data
    # hp.gnomview(onemap[i] - mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=1.2e0, 
    #             title=allTESNum[i], rot=center)
plt.show()
# sys.exit()

In [ ]:
plt.figure()
onemap_arr[onemap_arr==hp.UNSEEN] = 0
plt.imshow(onemap_arr)
plt.show()

In [ ]:
%%script echo skipping

# Same for all maps
nside = 256
azqubic = 116.4

### Check if the data has already been read from disk or not
try:
    data_TOD
except NameError:
    data_TOD = None
    

# allTESNum = [22, 26, 27, 28, 49, 50, 51, 52]
allTESNum = np.arange(256)+1
allmaps, data_TOD, center, newazt, newelt = pmf.make_coadded_maps(datadir, Obs_Site, allTESNum, data=data_TOD, 
                                       nside=nside, doplot=False, az_qubic=azqubic, parallel=True)


In [ ]:
%%script echo skipping

n0 = 8
for i in range(len(allTESNum)):
    ok = allmaps[i,:] != hp.UNSEEN
    if (i%(n0**2))==0 and i !=0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = allmaps[i,ok]-tdt.get_mode(allmaps[i,ok])
    mm[~ok] = hp.UNSEEN
    hp.gnomview(mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=2.5e4, 
                title=allTESNum[i], rot=center)
plt.show()

### We save the maps into a pickle file for subesquent usage.

In [ ]:
# pickle.dump( [allTESNum, allmaps], open( "NEW2025-allmaps-Jan14-2022.pkl", "wb" ) )
# pickle.dump( [allTESNum, allmaps], open( "NEW2025-allmaps-Jan14-2022_unturned.pkl", "wb" ) )

## We use saved maps from this point on

In [ ]:
# These are the visual inspection results
visibly_ok_arr = [False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
 False,  True,  True, False, False, False, False, False,  True, False, False, False,
 False, False, False,  True, False, False, False, False, False,  True, False, False,
  True, False,  True,  True, False, False, False, False,  True, False,  True, False,
  True,  True,  True,  True, False, False,  True, False,  True, False, False, False,
  True, False, False,  True, False, False, False, False,  True, False,  True, False,
 False,  True,  True, False, False, False, False, False, False, False,  True,  True,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False,  True,  True,  True, False,
  True, False, False,  True,  True,  True,  True, False, False, False, False, False,
 False, False, False, False, False,  True, False,  True, False, False, False, False,
 False,  True,  True, False, False, False, False, False, False, False, False, False,
 False,  True,  True, False, False, False, False,  True,  True,  True, False, False,
  True,  True, False, False, False, False, False,  True,  True, False, False, False,
  True,  True,  True, False, False, False,  True, False, False, False, False, False,
  True,  True,  True, False,  True,  True,  True,  True, False,  True,  True, False,
 False,  True, False, False,  True, False,  True, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
  True,  True,  True,  True, False, False,  True,  True, False,  True, False,  True,
 False, False, False, False,]

In [ ]:
# These are visibly OK but are actually ghosts
last_ghosts = [177, 214, 238, 241, 243, 244, 250]
for ghost in last_ghosts:
    visibly_ok_arr[ghost - 1] = False

## Gaussian peak fitting

In [ ]:
# %%script echo skipping
# 
# # allTESNum, allmaps = pickle.load( open( "NEW2025-allmaps-Jan14-2022.pkl", "rb" ) )
# We get the maps
allTESNum, allmaps = pickle.load( open( "NEW2025-allmaps-Jan14-2022_unturned.pkl", "rb" ) )

In [ ]:
plt.figure()
# gnommap = hp.gnomview(allmaps[151], reso=5, min=-5e3, max=1.2e4, 
#                 title="152", return_projected_map=True).data
gnommap = hp.gnomview(allmaps[151], reso=5, 
                title="152", return_projected_map=True).data
plt.show()

plt.figure()
gnommap[gnommap==hp.UNSEEN] = 0
plt.imshow(gnommap)
plt.show()

In [ ]:
# %%script echo skipping

# We perform the Moon position fit and show the result for one TES

### Pick a TES number
mynum = 152
xs = 201
# myrotinit = np.array([-1, 4, 0]) # np.array([0, 0, 0]) # np.array([-1, -4., 0])
myrotinit = np.array([0, 0, 0])
# myrotinit = np.array([-0.5, -4.5, 0])

idx = np.where(np.array(allTESNum)==mynum)[0][0]
#hp.gnomview(allmaps[idx,:], reso=10, rot=myrotinit, title='TES#{}'.format(mynum), min=-5e3, max=2.5e4, sub=(1,4,1))
# reso=4
reso=5

pack = scantype, newazt, newelt, nside

resfit, mapxy, mapfit, extent, fig = pmf.fit_one_tes(allmaps[idx,:], xs, reso, rot=myrotinit, verbose=True, doplot=True,
                                                    renorm=True, return_images=True, pack=pack)

plt.show()
sys.exit()

In [ ]:
resfit, mapxy, mapfit, extent, fig

In [ ]:
ipos = (np.arange(xs) - (xs - 1)/2)*reso/60
jpos = -ipos.copy()
ipos += myrotinit[1]
jpos -= myrotinit[0]

Ni, Nj = np.shape(mapxy)

In [ ]:
nsig = 1
mm, ss = ft.meancut(mapxy, 3)
mini = mm-nsig*ss
maxi = np.max(mapxy)

In [ ]:
m = resfit

In [ ]:
from scipy.fft import fft2
Nx = len(mapxy)
Ny = len(mapxy[0])
lobe_pos = (Nx//2, Ny//2)
Kx, Ky, K = pmf.get_K(Nx, Ny)
ft_phase = pmf.get_ft_phase(lobe_pos, Nx, Ny)
cos_win = pmf.cos_window(Nx, Ny, lx=20, ly=20)
deltaK = 1
Kbin = pmf.get_Kbin(deltaK, K)
nKbin = len(Kbin) - 1  # nb of bins
Kcent = (Kbin[:-1] + Kbin[1:])/2
size_pix = reso/60 # degree
# reso_instr = 0.92 # degree
reso_img = 1.036 # degree # test
ft_shape = fft2(pmf.gauss2D(Nx, Ny, x0=lobe_pos[0], y0=lobe_pos[1], reso=[reso_img/size_pix], normal=True))

# Test with a filtered Gaussian
# pars = 1, iipos[Nx//2, Ny//2], jjpos[Ny//2, Ny//2], reso_img/conv_reso_fwhm
# myfiltgauss_flat = g2d(0, pars)
# myfiltgauss = myfiltgauss_flat.reshape(Nx, Ny)
# ft_shape = fft2(myfiltgauss)

# plt.figure()
# plt.imshow(myfiltgauss)
# plt.show()

filtmapsn = pmf.get_filtmapsn(mapxy * cos_win, nKbin, K, Kbin, Kcent, ft_shape, ft_phase)

iipos, jjpos = np.meshgrid(ipos, jpos, indexing="ij")
maxii = filtmapsn == np.nanmax(filtmapsn)
max_i = np.mean(iipos[maxii])
max_j = np.mean(jjpos[maxii])
guess = np.array([1e4, max_i, max_j, reso_img/pmf.conv_reso_fwhm])

print(iipos[maxii], jjpos[maxii])
pos_max = np.argmax(maxii)
print(pos_max)
pos_maxi = pos_max % Ni
pos_maxj = pos_max // Ni % Nj

print(pos_maxi, pos_maxj)


plt.figure()
plt.imshow(filtmapsn)
plt.axvline(x=maxii[0])
plt.axhline(y=maxii[1])
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 4, width_ratios=(1, 1, 1, 0.05), figsize=(16, 5))
axs = pmp.plot_fit_img(mapxy, axs, ipos, jpos, iguess=guess[1], jguess=guess[2], ifit=m.values[1], jfit=m.values[2], vmin=mini, vmax=maxi, ms=10)
axs[1].imshow(mapfit, extent=[np.min(ipos), np.max(ipos), np.min(jpos), np.max(jpos)], vmin=mini, vmax=maxi)
im = axs[2].imshow(mapxy - mapfit, extent=[np.min(ipos), np.max(ipos), np.min(jpos), np.max(jpos)], vmin=mini, vmax=maxi)
axs[0].set_ylabel('Degrees')
for i in range(3):
    axs[i].set_xlabel('Degrees')
axs[2].set_title('Residuals')
plt.show()
print(guess)

In [ ]:
print(1, iipos[Nx//2 + 1, Ny//2], jjpos[Ny//2, Ny//2], reso_img/pmf.conv_reso_fwhm)

In [ ]:
### Let's try to plot the guessed map --> why is the guess so bad?
pars = m.values[0], guess[1], guess[2], reso_img/pmf.conv_reso_fwhm
g2d = pmf.filtgauss2dfit(ipos, jpos, scantype, newelt, newazt, nside)
myfiltgauss_flat = g2d(0, pars)
myfiltgauss = myfiltgauss_flat.reshape(Nx, Ny)

fig, axs = plt.subplots(1, 4, width_ratios=(1, 1, 1, 0.05), figsize=(16, 5))
axs = pmp.plot_fit_img(mapxy, axs, ipos, jpos, iguess=guess[1], jguess=guess[2], ifit=m.values[1], jfit=m.values[2], vmin=mini, vmax=maxi, ms=10)
axs[1].imshow(myfiltgauss, extent=[np.min(ipos), np.max(ipos), np.min(jpos), np.max(jpos)], vmin=mini, vmax=maxi)
im = axs[2].imshow(mapxy - myfiltgauss, extent=[np.min(ipos), np.max(ipos), np.min(jpos), np.max(jpos)], vmin=mini, vmax=maxi)
axs[0].set_ylabel('Degrees')
for i in range(3):
    axs[i].set_xlabel('Degrees')
axs[2].set_title('Residuals')
plt.show()

In [ ]:
0.4398197304159835*pmf.conv_reso_fwhm

In [ ]:
# %%script echo skipping

# We then perform it for all TES and store the results

# print(allTESNum)
# print(np.shape(allmaps))

reso = 5 #4
xs = 201
allamp = np.zeros(len(allTESNum))
allerramp = np.zeros(len(allTESNum))
allFWHM = np.zeros(len(allTESNum)) * np.nan
allerrFWHM = np.zeros(len(allTESNum))
allxy = np.zeros((len(allTESNum), 2)) * np.nan
allerrxy = np.zeros((len(allTESNum), 2))

moon_fit = []

for i in range(len(allTESNum)):
    # idx = np.where(np.array(allTESNum-1)==i)[0][0]
    if not visibly_ok_arr[i]:
        moon_fit.append([(np.nan, np.nan), np.nan, np.nan])
        continue
    m = pmf.fit_one_tes(allmaps[i,:], xs, reso, rot=myrotinit, verbose=False, renorm=True, doplot=False)
    allFWHM[i] = m.values[3] * pmf.conv_reso_fwhm
    allerrFWHM[i] = m.errors[3] * pmf.conv_reso_fwhm
    allamp[i] = m.values[0] * pmf.conv_reso_fwhm
    allerramp[i] = m.errors[0] * pmf.conv_reso_fwhm
    allxy[i, :] = m.values[1:3]
    allerrxy[i, :] = m.errors[1:3]
    print('TES#{0}: FWHM = {1:5.2f}'.format(i + 1, m.values[3] * pmf.conv_reso_fwhm))
    moon_fit.append([allxy[i, :], allFWHM[i], allamp[i]])

## Get offsets

We use DBSCAN to associate measured peaks to the ones predicted by creidhe

**there is an inversion somewhere between az and el... it has to be understood. It has been implemented here.**



In [ ]:
%%script echo skipping

#### Offsets from Créidhe
# import pickle
offsets = pickle.load( open( 'pointing_offsets_fixed_hole.pickle', 'rb') )
print(np.shape(offsets))

mytesn = np.array(allTESNum)
xycreidhe = offsets[mytesn -1 , :]
print(np.shape(xycreidhe))

######## Here we apply the inversion betwwen Az and El ##############
invert_azel = True
if invert_azel == True:
    xycreidhe = np.flip(xycreidhe, axis=1)
# print(xycreidhe[0])

invert_az = True
if invert_az == True:
    xycreidhe[:, 0] = -xycreidhe[:, 0]
# print(xycreidhe[0])

allTESNum = np.arange(256) + 1


In [ ]:
%%script echo skipping

print(xycreidhe)
# We try to find a cluster of points in the diff between real and predicted position of the Moon in order to find a shift of position
DBscan_ok = pmf.get_DBscan_res(allxy[:, 0], allxy[:, 1], xycreidhe[:, 0], xycreidhe[:, 1], allerrxy[:, 0], allerrxy[:, 1], allFWHM, allerrFWHM, visibly_ok_arr, doplot=True, eps=1, min_samples=10)
# DBscan_ok = get_DBscan_res(allxy[:, 0], allxy[:, 1], xycreidhe[:, ind_creidhe[0]], -xycreidhe[:, ind_creidhe[1]], allerrxy[:, 0], allerrxy[:, 1], allFWHM, allerrFWHM, visibly_ok_arr, doplot=True, eps=1, min_samples=10)
# sys.exit()


In [ ]:

%%script echo skipping

print(xycreidhe[:,1], xycreidhe[:,0])

plt.figure()
plt.subplot().set_aspect(1)
# plt.figure()
plt.plot(xycreidhe[:,  0], xycreidhe[:,  1], 'ro', label="Creidhe's positions")
plt.plot(allxy[:,0], allxy[:,1], 'ko', label="fitted positions")
plt.xlabel('$az^{Moon}$')
plt.ylabel('$el^{Moon}$')
plt.legend()
plt.show()

In [ ]:
%%script echo skipping

# There used to be two different VI arrays (Moon visible and Moon well-fitted), they are now the same
isok_arr_dflt = visibly_ok_arr

In [ ]:
%%script echo skipping

# Fit not working anymore, to be fixed (probably an error that is NaN)

# fig = plt.gcf()
# plt.close(fig)
plt.figure()
### Here is a new version of the FWHM histogram with only the "good" TES
# a = plt.hist(allFWHM[DBscan_ok]) # DBscan def of good
a = plt.hist(allFWHM[isok_arr_dflt]) # VI def of good
xx = 0.5*(a[1][:-1]+a[1][1:])
yy = a[0]

def gauss(x, pars):
    return pars[0]*np.exp(-0.5*(x-pars[1])**2/pars[2]**2)

notzero = yy != 0
data = fit.Data(xx[notzero], yy[notzero], np.sqrt(yy[notzero]), gauss, pnames=['Amp', '$x_0$', 'σ'])

guess = np.array([len(xx), 1., 0.2])
# resfit = ft.do_minuit(xx[notzero], yy[notzero], np.sqrt(yy[notzero]), guess, verbose=True,
#                           functname=gauss)
m, ch2, ndf = data.fit_minuit(guess)


xxx = np.linspace(min(xx), max(xx), 1000)
data.plot()
#plot(xxx, gauss(xxx, m.values), label='FWHM = {0:5.2f} +/- {1:5.2f}'.format(m.values[1], m.values[2]), lw=3)

plt.xlabel('FWHM (deg.)')
plt.ylabel('Number of TES')
plt.title('Ntes = {}'.format((DBscan_ok).sum()))
plt.show()



### Below the inversion between az and el is obvious from the inversion required to Créidhe's points

In [ ]:
%%script echo skipping

fig, axs = plt.subplots(1, 2, figsize=(12, 7))

ax = axs[0]
ax.set_aspect(1)
ax.plot(xycreidhe[:, 0], xycreidhe[:, 1],'bo', alpha=0.2, label='Offsets from Créidhe')
# ax.plot(allxy[:,0], allxy[:,1], 'ro', alpha=0.2, label='All Moon peaks')
ax.errorbar(allxy[:,0], allxy[:,1],xerr=allerrxy[:, 0], yerr=allerrxy[:, 0], alpha=0.2, fmt='ro', label='All Moon peaks')
ax.plot(xycreidhe[DBscan_ok, 0], xycreidhe[DBscan_ok, 1],'bo', label='Offsets from Créidhe (DBSCAN selected)')
ax.plot(allxy[DBscan_ok, 0], allxy[DBscan_ok, 1], 'ro', label='All Moon peaks (DBSCAN selected)')
ax.legend()
ax.set_xlabel('$\Delta_{az}$ [deg.]')
ax.set_ylabel('$\Delta_{el}$ [deg.]')

delta_az = allxy[:, 0] - xycreidhe[:, 0]
delta_el = allxy[:, 1] - xycreidhe[:, 1]

maz, saz = ft.meancut(delta_az[DBscan_ok], 3)
mel, sel = ft.meancut(delta_el[DBscan_ok], 3)

ax = axs[1]
ax.set_aspect(1)
# ax.plot(xycreidhe[:, 0], xycreidhe[:, 1], 'go', alpha=0.1, label='Offsets from Créidhe')      ### Inversion needed here
scatter_creid_all = ax.scatter(xycreidhe[:, 0], xycreidhe[:, 1], c='b', alpha=0.1, label='Offsets from Créidhe')      ### Inversion needed here
# ax.plot(xycreidhe[DBscan_ok, 0], xycreidhe[DBscan_ok, 1],'bo', label='Offsets from Créidhe')                ### Inversion needed here
scatter_creid = ax.scatter(xycreidhe[DBscan_ok, 0], xycreidhe[DBscan_ok, 1], c='b', label='Offsets from Créidhe')                ### Inversion needed here
# ax.plot(allxy[DBscan_ok, 0] - maz, allxy[DBscan_ok, 1] - mel, 'ro', label='Shifted Moon peaks')
# eb_DB = ax.errorbar(allxy[DBscan_ok, 0] - maz, allxy[DBscan_ok, 1] - mel, xerr=allerrxy[DBscan_ok, 0], yerr=allerrxy[DBscan_ok, 0], fmt='ro', markersize=8, alpha=0.5, label='Shifted Moon peaks')
scatter_DB = ax.scatter(allxy[DBscan_ok, 0] - maz, allxy[DBscan_ok, 1] - mel, c='r', s=80, alpha=0.8, label='Shifted Moon peaks')
# eb_VI = ax.errorbar(allxy[isok_arr_dflt, 0] - maz, allxy[isok_arr_dflt, 1] - mel, xerr=allerrxy[isok_arr_dflt, 0], yerr=allerrxy[isok_arr_dflt, 0], fmt='go', label='Shifted Moon peaks VI')
scatter_VI = ax.scatter(allxy[isok_arr_dflt, 0] - maz, allxy[isok_arr_dflt, 1] - mel, c='g', s=26, label='Shifted Moon peaks VI')
ax.set_title('Shift: $\Delta$Az={0:5.2f} +/- {1:5.2f} deg ; $\Delta$El={2:5.2f} +/- {3:5.2f} deg'.format(maz, saz, mel, sel))
ax.legend()
ax.set_xlabel('$\Delta_{az}$ [deg.]')
ax.set_ylabel('$\Delta_{el}$ [deg.]')

# plt.show()

names_VI = allTESNum[isok_arr_dflt]
names_DB = allTESNum[DBscan_ok]
names_creid = allTESNum[DBscan_ok]

scatter_list = [scatter_creid, scatter_DB, scatter_VI]
scatt_name_list = ["creid", "DB", "VI"]
all_names = [names_creid, names_DB, names_VI]

pmp.hover_cursor(fig, axs[1], scatter_list, scatt_name_list, all_names)
# hover_cursor_alpha(fig, axs[1], scatter_list, scatt_name_list, all_names)


In [ ]:
%%script echo skipping

fig, ax = plt.subplots(1, 1)

ax.set_aspect(1)
scatter_creid_all = ax.scatter(xycreidhe[:, 0], xycreidhe[:, 1], c='b', alpha=0.1)      ### Inversion needed here
scatter_creid = ax.scatter(xycreidhe[visibly_ok_arr, 0], xycreidhe[visibly_ok_arr, 1], c='b', label='Offsets from Créidhe')      ### Inversion needed here
scatter_DB = ax.scatter(allxy[visibly_ok_arr, 0] - maz, allxy[visibly_ok_arr, 1] - mel, c='r', s=80, alpha=0.8, label='Shifted Moon peaks')
# eb_VI = ax.errorbar(allxy[isok_arr_dflt, 0] - maz, allxy[isok_arr_dflt, 1] - mel, xerr=allerrxy[isok_arr_dflt, 0], yerr=allerrxy[isok_arr_dflt, 0], fmt='go', label='Shifted Moon peaks VI')
scatter_VI = ax.scatter(allxy[isok_arr_dflt, 0] - maz, allxy[isok_arr_dflt, 1] - mel, c='g', s=26, label='Shifted Moon peaks VI')
ax.set_title('Shift: $\Delta$Az={0:5.2f} +/- {1:5.2f} deg ; $\Delta$El={2:5.2f} +/- {3:5.2f} deg'.format(maz, saz, mel, sel))
ax.legend()
ax.set_xlabel('$\Delta_{az}$ [deg.]')
ax.set_ylabel('$\Delta_{el}$ [deg.]')

# plt.show()

names_VI = allTESNum[isok_arr_dflt]
names_DB = allTESNum[visibly_ok_arr]
names_creid = allTESNum[visibly_ok_arr]

scatter_list = [scatter_creid, scatter_DB, scatter_VI]
scatt_name_list = ["creid", "DB", "VI"]
all_names = [names_creid, names_DB, names_VI]

pmp.hover_cursor_alpha(fig, ax, scatter_list, scatt_name_list, all_names)


In [ ]:
%%script echo skipping

### Now apply it
nn = DBscan_ok.sum()
initvec = np.zeros((nn, 2))
outvec = np.zeros((nn, 2))
weightvec = np.zeros(nn)

initvec = np.zeros_like(xycreidhe[DBscan_ok,:])
initvec[:,0] = xycreidhe[DBscan_ok, 0]
initvec[:,1] = xycreidhe[DBscan_ok, 1]

outvec = allxy[DBscan_ok,:]
weightvec[:] = 1./np.sum(allerrxy[DBscan_ok]**2, axis=1)

fig, ax = plt.subplots(1, 1)
ax.set_aspect(1)
ax.plot(initvec[:,0], initvec[:,1], 'bo', label = 'Créidhe')
ax.plot(outvec[:,0], outvec[:,1], 'ro', label='Moon')
ax.plot(allxy[isok_arr_dflt, 0], allxy[isok_arr_dflt, 1], 'go', label='Moon VI')

guess = np.array([0., maz, mel, 1.])
print(guess)

data = fit.Data(np.ravel(initvec), np.ravel(outvec), np.ravel(initvec)*0+1, pmf.rot_trans_scale_pts)
m, ch2, ndf = data.fit_minuit(guess, fixpars=[3])

newvec = np.reshape(pmf.rot_trans_scale_pts(np.ravel(initvec), m.values), (nn,2))

mylabel = 'Fit to match Moon: \n'+ r'$\theta$={0:3.2f}+/-{1:3.2f} deg'.format(m.values[0], m.errors[0])
mylabel += '\n' + r'$\Delta$az={0:3.2f}+/-{1:3.2f} deg'.format(m.values[1], m.errors[1])
mylabel += '\n' + r'$\Delta$el={0:3.2f}+/-{1:3.2f} deg'.format(m.values[2], m.errors[2])
ax.plot(newvec[:,0], newvec[:,1], 'b+', ms=13, label=mylabel)
ax.legend()
ax.set_xlabel('$\Delta_{az}$ [deg.]')
ax.set_ylabel('$\Delta_{el}$ [deg.]')
plt.show()

In [ ]:
%%script echo skipping

# Not very useful plot anymore

ivec = np.zeros_like(xycreidhe[DBscan_ok, :]) # ivec comprends l'inversion az el
ivec[:,0] = xycreidhe[DBscan_ok, 0]
ivec[:,1] = xycreidhe[DBscan_ok, 1]

crei_rot = np.reshape(pmf.rot_trans_scale_pts(np.ravel(ivec), m.values), (len(allxy[DBscan_ok, 0]), 2))
pmf.assign_TES(allxy[DBscan_ok, 0], allxy[DBscan_ok, 1], crei_rot[:, 0], crei_rot[:, 1], tolerance=0.4, doplot=True)

In [ ]:
%%script echo skipping

print(np.sum(visibly_ok_arr))

## Visual inspection

In [ ]:
%%script echo skipping

from matplotlib.widgets import Button

# fig, axs = plt.subplots(1, 4, figsize=(10, 3), width_ratios=(1, 1, 1, 0.05))
fig, axs = plt.subplots(1, 2, figsize=(7, 7), width_ratios=(1, 0.05))

xs = 201
reso = 6

class Index:

    def __init__(self, TES_num, fig, axes, prev_selec, verbose=False):
        self.fig = fig
        self.prev_selec = prev_selec
        self.isok_arr = np.zeros(len(TES_num), dtype=bool)
        self.ind = -1
        self.axs = axes
        self.verbose = verbose
        self.goNext()

    
    def test_ind(self):
        if self.ind >= len(self.isok_arr):
            if self.verbose:
                print("Not good index")
            return False
        elif self.prev_selec[self.ind] == False:
            if self.verbose:
                print("Already rejected TES")
            self.isok_arr[self.ind] = False
            self.goNext()
        else:
            return True
    
    def goNext(self):
        self.ind += 1
        if self.verbose:
            print(self.ind)
        if not self.test_ind():
            return
        if self.verbose:
            print("TES {}".format(self.ind + 1))
        # _, _, _, _, _ = pmf.fit_one_tes(allmaps[self.ind,:], xs, reso, rot=myrotinit, return_images=True, renorm=True, doplot=True, axs=self.axs, xycreid_corr=crei_rot[self.ind, :], verbose=self.verbose)
        _, _, _, _, _ = pmf.fit_one_tes(allmaps[self.ind,:], xs, reso, rot=myrotinit, return_images=True, renorm=True, doplot=True, axs=self.axs, xycreid_corr=None, verbose=self.verbose)
        plt.subplots_adjust(hspace=0)
        self.fig.suptitle("TES {}".format(self.ind + 1))
        plt.draw()

    def good(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = True
        self.goNext()


    def bad(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = False
        self.goNext()


allTESNum = np.arange(256) + 1
callback = Index(allTESNum, fig, axs, visibly_ok_arr, verbose=False)
axbad = fig.add_axes([0.7, 0.05, 0.1, 0.075])
axgood = fig.add_axes([0.81, 0.05, 0.1, 0.075])
bgood = Button(axgood, 'Good')
bgood.on_clicked(callback.good)
bbad = Button(axbad, 'Bad')
bbad.on_clicked(callback.bad)

plt.show()

In [ ]:
%%script echo skipping

print(callback.isok_arr)

In [ ]:
%%script echo skipping

isok_arr_dflt = [False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True,  True, False,
 False,  True,  True,  True, False, False, False, False,  True,  True, False, False,
 False, False, False,  True, False, False, False, False, False,  True, False, False,
 False, False,  True,  True, False,  True, False, False,  True, False,  True, False,
  True,  True,  True,  True, False, False,  True, False,  True,  True, False,  True,
  True, False, False,  True, False, False,  True,  True, False, False,  True, False,
 False,  True,  True,  True, False, False, False, False, False,  True,  True,  True,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False,  True,  True, False,  True, False, False,  True, False, False,
 False, False, False, False, False, False, False, False,  True,  True,  True, False,
  True, False,  True,  True,  True,  True,  True, False, False, False, False, False,
 False,  True, False,  False, False,  True, False,  True,  True, False, False, False,
 False,  True, False, False, False,  True,  True, False, False, False, False, False,
 False,  True,  True, False,  True, False,  True,  True,  True,  True, False, False,
  True,  True, False,  True,  True, False,  True,  True,  True, False, False, False,
  True,  True,  True, False, False, False,  True,  True, False, False, False, False,
  True,  True,  True, False, False,  True,  True,  True, False,  True, False,  True,
 False,  True, False, False,  True, False,  True, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
  True,  True,  True,  True, False,  True,  True,  True, False,  True, False,  True,
 False, False, False, False]

In [ ]:
%%script echo skipping

# We perform the Moon position fit and show the result for one TES

### Pick a TES number
mynum = 133
xs = 201
# myrotinit = np.array([-1, 4, 0]) # np.array([0, 0, 0]) # np.array([-1, -4., 0])
myrotinit = np.array([0, 0, 0])
# myrotinit = np.array([-0.5, -4.5, 0])

idx = np.where(np.array(allTESNum)==mynum)[0][0]
#hp.gnomview(allmaps[idx,:], reso=10, rot=myrotinit, title='TES#{}'.format(mynum), min=-5e3, max=2.5e4, sub=(1,4,1))
# reso=4
reso=5

resfit, mapxy, mapfit, extent, fig = pmf.fit_one_tes(allmaps[idx,:], xs, reso, rot=myrotinit, verbose=True, doplot=True,
                                                    renorm=True, return_images=True)

plt.show()

In [ ]:
%%script echo skipping

# We perform the Moon position fit and show the result for one TES

### Pick a TES number
mynum = 133
# xs = 201
xs = 301
# myrotinit = np.array([-1, 4, 0]) # np.array([0, 0, 0]) # np.array([-1, -4., 0])
myrotinit = np.array([0, 0, 0])
# myrotinit = np.array([-0.5, -4.5, 0])

idx = np.where(np.array(allTESNum)==mynum)[0][0]
#hp.gnomview(allmaps[idx,:], reso=10, rot=myrotinit, title='TES#{}'.format(mynum), min=-5e3, max=2.5e4, sub=(1,4,1))
# reso=4
reso=5

resfit, mapxy, mapfit, extent, fig = pmf.fit_one_tes_sb(allmaps[idx,:], xs, reso, rot=myrotinit, verbose=True, doplot=True,
                                                    renorm=True, return_images=True)

plt.show()

In [ ]:
synthbeam_list, qubic_dict = pmf.get_synthbeam_freqs(idet=mynum, nsub=16, nside=nside) # array of the synthesized beams from the instrument for nsub//2 freqs
tot_sb = np.sum(synthbeam_list, axis=0)

In [ ]:
print(qubic_dict.keys())
print(qubic_dict["synthbeam_peak150_fwhm"])

In [ ]:
print(qubic_dict['filter_relative_bandwidth'])
bandwidth = qubic_dict['filter_relative_bandwidth']*150
print(150 - bandwidth/2, 150 + bandwidth/2)
sys.exit()

In [ ]:
from matplotlib.widgets import Button, Slider
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

class sliders_plot_sb():

    def __init__(self, in_map, synthbeam):
        self.reso = 5
        self.xs = 301
        self.nside = int(np.sqrt(len(in_map)/12))
        obs_fwhm = 0.92 # deg
        # moon_size = 0.5 # deg# So instr_fwhm should be np.sqrt(obs_fwhm**2 - moon_size**2) = 0.77 deg? 
        FI_fwhm = 0.39268176 # deg, at 150 GHz
        corr_fwhm = np.sqrt(obs_fwhm**2 - FI_fwhm)
        print(corr_fwhm)
        self.Conv_moon = HealpixConvolutionGaussianOperator(fwhm=corr_fwhm*np.pi/180, lmax=2 * self.nside) # The Moon is not exactly a point source
        
        self.synthbeam = self.Conv_moon(synthbeam)
        # self.synthbeam = synthbeam
        rotator = hp.rotator.Rotator(rot=(0, 90, 0), eulertype="ZYX", deg=True)
        # rotator = hp.rotator.Rotator(rot=(0, -90, 0), eulertype="ZYX", deg=True)
        self.synthbeam_rot = rotator.rotate_map_alms(self.synthbeam)

        mm = in_map#.copy()
        badpix = mm == hp.UNSEEN
        mm[badpix] = 0          ### Set bad pixels to zero before returning the np.array()
        self.in_map_proj = hp.gnomview(mm, reso=self.reso, rot=[0, 0, 0], return_projected_map=True, xsize=self.xs, no_plot=True).data
        # self.in_map = in_map.copy()

        self.init_pos()
        self.create_fig()
        self.add_sliders()
        self.init_plot()
        self.get_reset()

        # register the update function with each slider
        for slider in self.list_sliders:
            slider.on_changed(self.update)

        self.button.on_clicked(self.reset)

        self.reset(True)

    # Define initial parameters

    def init_pos(self):
        # self.rot = np.array([0, 90, 90])
        # self.rot = np.array([2.4, 6.7, 1])
        # self.rot = np.array([0.4, 7.4, 91.5])
        self.rot = np.array([4.5, 4.5, 87]) # TES 152?
        # self.rot = np.array([0, -89.5, 97.3]) # TES 152 X?
        # self.rot = np.array([83.5, 89, 1.5]) # Y
        # self.rot = np.array([0, -89.5, 97.3]) # X
        # self.rot = np.array([1.5, 1.5, 83])
        self.init_translation = self.rot[:-1]
        self.init_rotation = self.rot[-1]
        self.init_amplitude = 70

    def create_fig(self):
        # Create the figure and the line that we will manipulate
        self.fig, self.axs = plt.subplots(1, 3, figsize=(14, 6))

    def init_plot(self):
        self.update_sb_map()

        ax = self.axs[0]
        self.img0 = ax.imshow(self.in_map_proj)

        ax = self.axs[1]
        self.img1 = ax.imshow(self.sb_map)
        # self.img1 = ax.imshow(self.in_map_proj)

        ax = self.axs[2]
        self.img2 = ax.imshow(self.in_map_proj - self.sb_map)
        # self.img2 = ax.imshow(self.in_map_proj)

    def add_sliders(self):
        find_pos = True
        delta_deg = 10
        # adjust the main plot to make room for the sliders
        self.fig.subplots_adjust(left=0.25, bottom=0.25)

        # Make two horizontal slider to control the translation.
        # axtransaz = self.fig.add_axes([0.25, 0.15, 0.65, 0.03])
        # axtransel = self.fig.add_axes([0.25, 0.1, 0.65, 0.03])

        self.transaz_slider = self.init_rot_slider(init_pos=self.init_translation[0], delta_deg=delta_deg,
                                                    axpos=[0.25, 0.15, 0.65, 0.03], find_pos=find_pos, orientation="horizontal",
                                                    label="Rot1 [deg]")
       
        # init_pos = self.init_translation[0]
        # if find_pos:
        #     vals = [-180, 180]
        # else:
        #     vals = [init_pos - delta_deg, init_pos + delta_deg]
        
        # self.transaz_slider = Slider(
        #     ax=axtransaz,
        #     label='Rot1 [deg]',
        #     valmin=vals[0],
        #     valmax=vals[1],
        #     valinit=init_pos,
        # )

        self.transel_slider = self.init_rot_slider(init_pos=self.init_translation[1], delta_deg=delta_deg,
                                                    axpos=[0.25, 0.1, 0.65, 0.03], find_pos=find_pos, orientation="horizontal",
                                                    label="Rot2 [deg]")
        
        # init_pos = self.init_translation[1]
        # if find_pos:
        #     vals = [-180, 180]
        # else:
        #     vals = [init_pos - delta_deg, init_pos + delta_deg]
        # self.transel_slider = Slider(
        #     ax=axtransel,
        #     label='Rot2 [deg]',
        #     valmin=vals[0],
        #     valmax=vals[1],
        #     valinit=init_pos,
        # )

        # Make a vertically oriented slider to control the amplitude
        axamp = self.fig.add_axes([0.15, 0.25, 0.0225, 0.63])
        self.amp_slider = Slider(
            ax=axamp,
            label="Scale",
            valmin=0.2,
            valmax=120,
            valinit=self.init_amplitude,
            orientation="vertical"
        )


        # Make a vertically oriented slider to control the rotation
        self.rotation_slider = self.init_rot_slider(init_pos=self.init_rotation, delta_deg=delta_deg,
                                                    axpos=[0.1, 0.25, 0.0225, 0.63], find_pos=find_pos, orientation="vertical",
                                                    label="Rot3 [deg]")
        # init_pos = self.init_rotation
        # if find_pos:
        #     vals = [-180, 180]
        # else:
        #     vals = [init_pos - delta_deg, init_pos + delta_deg]
        # axrot = self.fig.add_axes([0.1, 0.25, 0.0225, 0.63])
        # self.rotation_slider = Slider(
        #     ax=axrot,
        #     label="Rot3 [deg]",
        #     valmin=vals[0],
        #     valmax=vals[1],
        #     valinit=init_pos,
        #     orientation="vertical"
        # )

        self.list_sliders = [self.transaz_slider, self.transel_slider, self.amp_slider, self.rotation_slider]

    
    def init_rot_slider(self, init_pos, delta_deg, axpos, find_pos, orientation, label):
        if find_pos:
            vals = [-180, 180]
        else:
            vals = [init_pos - delta_deg, init_pos + delta_deg]
        axrot = self.fig.add_axes(axpos)
        slider = Slider(
            ax=axrot,
            label=label,
            valmin=vals[0],
            valmax=vals[1],
            valinit=init_pos,
            orientation=orientation
        )
        return slider

    def update_sb_map(self):
        # self.sb_map = hp.gnomview(self.synthbeam, reso=self.reso, rot=[self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val], min=-5e2, max=1e6, return_projected_map=True, xsize=self.xs, no_plot=True).data * self.amp_slider.val
        # Fonctionne mais signal non filtré --> trouver moyen de filtrer selft.synthbeam_rot sans rebouger la carte HEALPix
        self.sb_map = hp.gnomview(self.synthbeam_rot, reso=self.reso, rot=[self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val], min=-5e2, max=1e6, return_projected_map=True, xsize=self.xs, no_plot=True).data * self.amp_slider.val

        # rotator = hp.rotator.Rotator(rot=(self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val))
        # self.synthbeam_rot = rotator.rotate_map_alms(self.synthbeam)
        # self.sb_map = hp.gnomview(self.synthbeam_rot, reso=self.reso, rot=[0, 0, 0], min=-5e2, max=1e6, return_projected_map=True, xsize=self.xs, no_plot=True).data * self.amp_slider.val
        # print(np.max(self.sb_map))

        # newazt and newelt are computed at the start of this code
        # amp_azt = 
        # amp_elt = 
        # pmf.img_to_TOD(self.sb_map, amp_azt, amp_elt, newazt, newelt)

        # Fonctionne mais trop lent
        # rotator = hp.rotator.Rotator(rot=(self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val), eulertype="X", deg=True)
        # self.synthbeam_rot = rotator.rotate_map_alms(self.synthbeam) * self.amp_slider.val
        # fake_TOD = pmf.map_to_TOD(self.synthbeam_rot, newazt, newelt)
        # fake_TOD = pmf.my_filt(fake_TOD)
        # self.synthbeam_rot_filt, _ = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], fake_TOD[scantype != 0], nside=self.nside)
        # self.sb_map = hp.gnomview(self.synthbeam_rot_filt, reso=self.reso, rot=[0, 0, 0], min=-5e2, max=1e6, return_projected_map=True, xsize=self.xs, no_plot=True).data# * self.amp_slider.val

        # print(np.min(newazt), np.max(newazt))
        # print(np.min(newelt), np.max(newelt))

        # print(np.min(newazt - self.transaz_slider.val), np.max(newazt - self.transaz_slider.val))
        # print(np.min(newelt - self.transel_slider.val), np.max(newelt - self.transel_slider.val))
        # fake_TOD = pmf.map_to_TOD(self.synthbeam_rot, newazt, newelt)
        # fake_TOD = pmf.my_filt(fake_TOD)
        # self.synthbeam_rot_filt, _ = pmf.healpix_map(newazt[scantype != 0], newelt[scantype != 0], fake_TOD[scantype != 0], nside=self.nside)
        # self.sb_map = hp.gnomview(self.synthbeam_rot_filt, reso=self.reso, rot=[self.transaz_slider.val, self.transel_slider.val, self.rotation_slider.val], min=-5e2, max=1e6, return_projected_map=True, xsize=self.xs, no_plot=True).data * self.amp_slider.val
        

    # The function to be called anytime a slider's value changes
    def update(self, val):
        self.update_sb_map()
        self.img1.set_data(self.sb_map)
        self.img2.set_data(self.in_map_proj - self.sb_map)
        self.fig.canvas.draw_idle()

    def get_reset(self):
        # Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
        resetax = self.fig.add_axes([0.8, 0.025, 0.1, 0.04])
        self.button = Button(resetax, 'Reset', hovercolor='0.975')


    def reset(self, event):
        for slider in self.list_sliders:
            slider.reset()



In [ ]:
# mynum = 133
# idx = np.where(np.array(allTESNum)==mynum)[0][0]

sliders_plot_sb(onemap[0,:], 2e-5 * tot_sb)